# **SubSavers**

1. Problem - The average person wastes $400 annually on unused subscriptions. With the rise of digital services, managing multiple subscriptions has become overwhelming, leading to unnecessary expenses and cluttered finances.

2. Solution -  AI-powered subscription tool integrated into Nokia devices that tracks, analyzes, and optimizes users' subscriptions, saving them money by identifying and canceling underused services.

3. Key Features - Subscription Tracking:Automatically detects active subscriptions linked to the user’s accounts (e.g., credit cards, app stores) , Ai powered insights:Analyzes usage patterns (e.g., how often a service is accessed) and Flags underused or redundant subscriptions, One click-cancellation:Integrates with subscription platforms to allow users to cancel services directly from the app and Sends reminders before free trials convert to paid subscriptions.

4. How it works?
 - Users connect their payment methods (credit cards, PayPal, etc.) and app store accounts.
 - The app scans for active subscriptions and categorizes them. The AI analyzes usage data (e.g., app usage frequency, transaction history)
 - Identifies subscriptions with low engagements.
 - Then users recieve notifications like:  “You haven’t used [Service X] in 3 months. Save $10/month by canceling.”


In [2]:
import pandas as pd
from sklearn.cluster import KMeans
from datetime import datetime, timedelta

# Mock Data: Simulate user subscription data
data = {
    'Subscription': ['Netflix', 'Spotify', 'Gym Membership', 'Amazon Prime', 'New York Times'],
    'Subscription_ID': [101, 102, 103, 104, 105],
    'Monthly_Cost': [15, 10, 30, 12, 8],
    'Signup_Date': ['2023-01-01', '2023-03-15', '2023-02-10', '2023-04-01', '2023-05-20'],
    'Last_Used': ['2023-09-25', '2023-09-10', '2023-07-15', '2023-09-28', '2023-09-29'],
    'Usage_Frequency': [20, 5, 2, 10, 30]  # Days used in the last month
}

df = pd.DataFrame(data)
df

,Subscription,Subscription_ID,Monthly_Cost,Signup_Date,Last_Used,Usage_Frequency
0,Netflix,101,15,2023-01-01,2023-09-25,20
1,Spotify,102,10,2023-03-15,2023-09-10,5
2,Gym Membership,103,30,2023-02-10,2023-07-15,2
3,Amazon Prime,104,12,2023-04-01,2023-09-28,10
4,New York Times,105,8,2023-05-20,2023-09-29,30


In [5]:
df['Signup_Date'] = pd.to_datetime(df['Signup_Date'])
df['Last_Used'] = pd.to_datetime(df['Last_Used'])

# Calculate days since last used
df['Days_Since_Last_Used'] = (datetime.now() - df['Last_Used']).dt.days

df['Usage_Ratio'] = df['Usage_Frequency'] / 30

X = df[['Usage_Ratio', 'Monthly_Cost']] #clustering


kmeans = KMeans(n_clusters=2, random_state=0).fit(X) # to identify underused subscriptions
df['Cluster'] = kmeans.labels_

underused_subscriptions = df[df['Cluster'] == df['Cluster'].mode()[0]]
underused_subscriptions

,Subscription,Subscription_ID,Monthly_Cost,Signup_Date,Last_Used,Usage_Frequency,Days_Since_Last_Used,Usage_Ratio,Cluster
0,Netflix,101,15,2023-01-01,2023-09-25,20,495,0.666667,1
1,Spotify,102,10,2023-03-15,2023-09-10,5,510,0.166667,1
3,Amazon Prime,104,12,2023-04-01,2023-09-28,10,492,0.333333,1
4,New York Times,105,8,2023-05-20,2023-09-29,30,491,1.000000,1


In [6]:

# Generate Recommendations
def generate_recommendations(subscriptions):
    recommendations = []
    for _, row in subscriptions.iterrows():
        if row['Days_Since_Last_Used'] > 20:  # Not used in the last 20 days
            message = (
                f"Cancel Recommendation: You haven't used {row['Subscription']} in {row['Days_Since_Last_Used']} days. "
                f"Save ${row['Monthly_Cost']}/month."
            )
        else:
            message = (
                f"Usage Alert: You're using {row['Subscription']} {row['Usage_Frequency']} days/month. "
                f"Consider optimizing if usage drops further."
            )
        recommendations.append(message)
    return recommendations

recommendations = generate_recommendations(underused_subscriptions)

# Output Recommendations
print("Subscription Recommendations:")
for rec in recommendations:
    print(f"- {rec}")

# notifications
free_trial_end_date = datetime.now() + timedelta(days=2)
print(f"\nNotification: Your free trial for 'Fitness Pro' ends on {free_trial_end_date.strftime('%Y-%m-%d')}. Cancel now to avoid charges.")

Subscription Recommendations:
- Cancel Recommendation: You haven't used Netflix in 495 days. Save $15/month.
- Cancel Recommendation: You haven't used Spotify in 510 days. Save $10/month.
- Cancel Recommendation: You haven't used Amazon Prime in 492 days. Save $12/month.
- Cancel Recommendation: You haven't used New York Times in 491 days. Save $8/month.

Notification: Your free trial for 'Fitness Pro' ends on 2025-02-03. Cancel now to avoid charges.
